In [13]:
from StateTraceParser import StateTraceParser

In [14]:
parser = StateTraceParser("/Users/aditya/Documents/GitHub/game_creation_research/Object Physics Sandbox/Builds/ExperimenterView_21April22.app/Contents/InteractionLogs/logs_2022_04_23_14_37_32_game2_setup.json")

In [15]:
parser.df

,ball_x,ball_y,corner_x,corner_y,bucket_x,bucket_y,triangle_x,triangle_y,gear_x,gear_y,crate_x,crate_y,velocity_x,velocity_y,reset,note
0,-1.35,-4.12,5.34,-2.65,5.340000,1.850000,5.34,-1.25,5.34,0.24,5.340000,3.260000,0.0,0.0,0.0,
1,-1.35,-4.12,5.34,-2.65,5.340000,1.850000,5.34,-1.25,5.34,0.24,5.340000,3.260000,0.0,0.0,0.0,
2,-1.35,-4.12,5.34,-2.65,5.340000,1.850000,5.34,-1.25,5.34,0.24,5.340000,3.260000,0.0,0.0,0.0,
3,-1.35,-4.12,5.34,-2.65,5.340000,1.850000,5.34,-1.25,5.34,0.24,5.340000,3.260000,0.0,0.0,0.0,
4,-1.35,-4.12,5.34,-2.65,5.340000,1.850000,5.34,-1.25,5.34,0.24,5.340000,3.260000,0.0,0.0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,-1.35,-4.12,5.34,-2.65,-4.847445,-4.238384,5.34,-1.25,5.34,0.24,-1.262854,2.134222,0.0,0.0,0.0,
1413,-1.35,-4.12,5.34,-2.65,-4.847445,-4.238384,5.34,-1.25,5.34,0.24,-1.262854,2.134222,0.0,0.0,0.0,
1414,-1.35,-4.12,5.34,-2.65,-4.847445,-4.238384,5.34,-1.25,5.34,0.24,-1.262854,2.134222,0.0,0.0,0.0,
1415,-1.35,-4.12,5.34,-2.65,-4.847445,-4.238384,5.34,-1.25,5.34,0.24,-1.262854,2.134222,0.0,0.0,0.0,


In [18]:
parser.df[parser.df["note"]!=""]

,ball_x,ball_y,corner_x,corner_y,bucket_x,bucket_y,triangle_x,triangle_y,gear_x,gear_y,crate_x,crate_y,velocity_x,velocity_y,reset,note
903,-1.35,-4.12,5.34,-2.65,5.340000,1.850000,5.34,-1.25,5.34,0.24,-1.262854,2.134222,0.0,0.0,0.0,box pos\n
1343,-1.35,-4.12,5.34,-2.65,-4.847445,-4.238384,5.34,-1.25,5.34,0.24,-1.262854,2.134222,0.0,0.0,0.0,buvket pos\n
